In [29]:
import pandas as pd
import string
import demoji
demoji.download_codes()
import nltk
import re
from nltk.tokenize import word_tokenize
import contractions
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import seaborn as sns
import unicodedata
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

C:\Users\aman9\AppData\Local\Temp\ipykernel_2052\1369047291.py:4: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aman9\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aman9\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aman9\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Load Data

In [30]:
df =pd.read_csv("train.csv")
print(df.shape)
df.head()

(37500, 19)


,index,beer/ABV,beer/beerId,beer/brewerId,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,review/timeStruct,review/timeUnix,user/ageInSeconds,user/birthdayRaw,user/birthdayUnix,user/gender,user/profileName
0,40163,5.0,46634,14338,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,"{'min': 38, 'hour': 3, 'mday': 16, 'sec': 10, ...",1229398690,NaN,NaN,NaN,NaN,RblWthACoz
1,8135,11.0,3003,395,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,"{'min': 38, 'hour': 23, 'mday': 8, 'sec': 58, ...",1218238738,NaN,NaN,NaN,NaN,BeerSox
2,10529,4.7,961,365,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,"{'min': 7, 'hour': 18, 'mday': 26, 'sec': 2, '...",1101492422,NaN,NaN,NaN,Male,mschofield
3,44610,4.4,429,1,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,"{'min': 7, 'hour': 1, 'mday': 20, 'sec': 5, 'y...",1308532025,1.209827e+09,"Aug 10, 1976",208508400.0,Male,molegar76
4,37062,4.4,4904,1417,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,"{'min': 51, 'hour': 6, 'mday': 12, 'sec': 48, ...",1299912708,NaN,NaN,NaN,NaN,Brewbro000


# Check Nans

In [31]:
df.isnull().sum()/len(df)*100

index                 0.000000
beer/ABV              0.000000
beer/beerId           0.000000
beer/brewerId         0.000000
beer/name             0.000000
beer/style            0.000000
review/appearance     0.000000
review/aroma          0.000000
review/overall        0.000000
review/palate         0.000000
review/taste          0.000000
review/text           0.026667
review/timeStruct     0.000000
review/timeUnix       0.000000
user/ageInSeconds    79.050667
user/birthdayRaw     79.050667
user/birthdayUnix    79.050667
user/gender          59.162667
user/profileName      0.013333
dtype: float64

In [35]:
df.columns

Index(['beer/ABV', 'beer/name', 'beer/style', 'review/appearance',
       'review/aroma', 'review/overall', 'review/palate', 'review/taste',
       'review/text', 'user/gender'],
      dtype='object')

# Remove Redunadnat COLS

In [33]:
irrelevant_cols = ['index', 'beer/beerId', 'beer/brewerId', 'review/timeStruct', 'review/timeUnix', 
                   'user/birthdayRaw', 'user/birthdayUnix', 'user/profileName', 'user/ageInSeconds']
df.drop(columns=irrelevant_cols, inplace=True)

In [34]:
df.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender
0,5.0,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,NaN
1,11.0,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,NaN
2,4.7,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male
3,4.4,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male
4,4.4,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,NaN


#  Text Preprocessing

In [36]:
df['user/gender'].fillna("Unknown",inplace=True)
df['review/text'].fillna(" ",inplace=True)
df.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender
0,5.0,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown
1,11.0,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown
2,4.7,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male
3,4.4,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male
4,4.4,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown


In [37]:
%%time
def process_text(text):
        text = text.lower()
        text = re.sub(r"http\S+|www\S+|https\S+", "", text) # remove url
        text = re.sub(r'\([^)]*\)|\{[^}]*\}|\[[^\]]*\]', '', text) # remove brackets
        text = re.sub('<.*?>+', '', text) # remove html tags
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text) # remove punctuation
        text = re.sub(r"\d+", "", text) # Remove numeric characters
        text = re.sub(r'\s+', ' ', text) # Remove extra whitespace
        text=re.sub(r'#\w+', '', text) # Remove hasthtags
        text=re.sub("#[A-Za-z0-9_]+"," ",text) # Remove user mentions
        #text=re.sub(r"[^A-Za-z0-9\s]+", "", text) # Remove non alpha numeric
        text=re.sub(r'[₹\$\€\£\¥\₽]', '', text) # Remove currencies symbols
        text = contractions.fix(text) # Apply contractions
        text = re.sub(r'(.)\1{3,}',r'\1', text) # Remove repeated characters like helllooo to hello
        return text
    
def normalize_text(text):
    return ''.join(
        [c for c in unicodedata.normalize('NFKD', text)
         if unicodedata.category(c) != 'Mn']
    )

def remove_emojis_from_column(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].apply(lambda text: demoji.replace(text, ""))
    return dataframe

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word.lower() not in stop_words])




df['clean_text'] = df['review/text'].apply(normalize_text)    
df['clean_text'] = df['clean_text'].apply(process_text)
df = remove_emojis_from_column(df, 'clean_text')
df['clean_text']=df['clean_text'].apply(remove_stopwords)
df.head()

CPU times: total: 1min 29s
Wall time: 1min 29s


,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender,clean_text
0,5.0,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown,pours clouded gold thin white head nose quite ...
1,11.0,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown,oz bottle oz snifter deep ruby red hue one fin...
2,4.7,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male,first enjoyed brewpub years ago finally manage...
3,4.4,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male,first thing noticed pouring green bottle glass...
4,4.4,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown,pours amber one finger head strong pour head r...


In [38]:
df['clean_text'][0]

'pours clouded gold thin white head nose quite floral larger amount spices added definitely spice forward fragrance flavor odd burn hits first sip fades seems like dirty vanilla aftertaste perhaps absinthe regardless get quite spiced tone tongue almost feel little heat think inexperienced palate spices contributing ignorance precisely overall nice drinker indeed'

# Remopve Stopwords

In [39]:

df.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender,clean_text
0,5.0,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown,pours clouded gold thin white head nose quite ...
1,11.0,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown,oz bottle oz snifter deep ruby red hue one fin...
2,4.7,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male,first enjoyed brewpub years ago finally manage...
3,4.4,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male,first thing noticed pouring green bottle glass...
4,4.4,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown,pours amber one finger head strong pour head r...


In [40]:
len(df['beer/style'].unique())

95

In [41]:
len(df['beer/name'].unique())

1688

# Feature Engineering

In [42]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
##1. Feature Exctraction -- Convert sentecne into vectr embed
tfidf = TfidfVectorizer(stop_words='english', max_features=500)
text_features = tfidf.fit_transform(df['clean_text']).toarray()

text_features_df = pd.DataFrame(text_features, columns=tfidf.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), text_features_df], axis=1)

df.head()

,beer/ABV,beer/name,beer/style,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender,...,wood,woody,work,worth,wow,year,years,yeast,yeasty,yellow
0,5.0,Chiostro,Herbed / Spiced Beer,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,11.0,Bearded Pat's Barleywine,American Barleywine,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,4.7,Naughty Nellie's Ale,American Pale Ale (APA),3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.259233,0.0,0.0,0.0
3,4.4,Pilsner Urquell,Czech Pilsener,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,4.4,Black Sheep Ale (Special),English Pale Ale,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


Index(['beer/ABV', 'review/appearance', 'review/aroma', 'review/overall',
       'review/palate', 'review/taste', 'review/text', 'user/gender',
       'clean_text', 'abbey',
       ...
       'wood', 'woody', 'work', 'worth', 'wow', 'year', 'years', 'yeast',
       'yeasty', 'yellow'],
      dtype='object', length=25511)

In [43]:
##2. Frequency Encoding
name_counts = df['beer/name'].value_counts()
style_cnts=df['beer/style'].value_counts()
df['beer/name_freq'] = df['beer/name'].map(name_counts)
df['beer/style_freq']=df['beer/style'].map(style_cnts)
df.drop(columns=['beer/style', 'beer/name'], inplace=True)

df.head()

,beer/ABV,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender,clean_text,absolutely,...,work,worth,wow,year,years,yeast,yeasty,yellow,beer/name_freq,beer/style_freq
0,5.0,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown,pours clouded gold thin white head nose quite ...,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,17,62
1,11.0,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown,oz bottle oz snifter deep ruby red hue one fin...,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,61,634
2,4.7,3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male,first enjoyed brewpub years ago finally manage...,0.0,...,0.0,0.0,0.0,0.0,0.259233,0.0,0.0,0.0,78,1711
3,4.4,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male,first thing noticed pouring green bottle glass...,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,947,1131
4,4.4,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown,pours amber one finger head strong pour head r...,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,252,971


In [44]:
###2. Convert Gender to Numerical Encoding
one_hot = OneHotEncoder(handle_unknown='ignore', sparse=False)
encoded_features = one_hot.fit_transform(df['user/gender'].values.reshape(-1, 1))
encoded_df = pd.DataFrame(encoded_features, columns=one_hot.get_feature_names_out())
df = pd.concat([df.reset_index(drop=True), encoded_df.reset_index(drop=True)], axis=1)
df.head()

C:\Users\aman9\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,beer/ABV,review/appearance,review/aroma,review/overall,review/palate,review/taste,review/text,user/gender,clean_text,absolutely,...,year,years,yeast,yeasty,yellow,beer/name_freq,beer/style_freq,x0_Female,x0_Male,x0_Unknown
0,5.0,4.0,4.0,4.0,4.0,4.0,Pours a clouded gold with a thin white head. N...,Unknown,pours clouded gold thin white head nose quite ...,0.0,...,0.0,0.000000,0.0,0.0,0.0,17,62,0.0,0.0,1.0
1,11.0,4.0,3.5,3.5,3.5,3.0,12oz bottle into 8oz snifter.\t\tDeep ruby red...,Unknown,oz bottle oz snifter deep ruby red hue one fin...,0.0,...,0.0,0.000000,0.0,0.0,0.0,61,634,0.0,0.0,1.0
2,4.7,3.5,4.0,3.5,3.5,3.5,First enjoyed at the brewpub about 2 years ago...,Male,first enjoyed brewpub years ago finally manage...,0.0,...,0.0,0.259233,0.0,0.0,0.0,78,1711,0.0,1.0,0.0
3,4.4,3.0,3.0,2.5,3.0,3.0,First thing I noticed after pouring from green...,Male,first thing noticed pouring green bottle glass...,0.0,...,0.0,0.000000,0.0,0.0,0.0,947,1131,0.0,1.0,0.0
4,4.4,4.0,3.0,3.0,3.5,2.5,A: pours an amber with a one finger head but o...,Unknown,pours amber one finger head strong pour head r...,0.0,...,0.0,0.000000,0.0,0.0,0.0,252,971,0.0,0.0,1.0


In [48]:
ll = df.drop(columns=['review/overall','user/gender','review/text','clean_text'])
ll.head()

,beer/ABV,review/appearance,review/aroma,review/palate,review/taste,absolutely,abv,actually,adds,aftertaste,...,year,years,yeast,yeasty,yellow,beer/name_freq,beer/style_freq,x0_Female,x0_Male,x0_Unknown
0,5.0,4.0,4.0,4.0,4.0,0.0,0.0,0.0,0.0,0.158814,...,0.0,0.000000,0.0,0.0,0.0,17,62,0.0,0.0,1.0
1,11.0,4.0,3.5,3.5,3.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,61,634,0.0,0.0,1.0
2,4.7,3.5,4.0,3.5,3.5,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.259233,0.0,0.0,0.0,78,1711,0.0,1.0,0.0
3,4.4,3.0,3.0,3.0,3.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,947,1131,0.0,1.0,0.0
4,4.4,4.0,3.0,3.5,2.5,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,252,971,0.0,0.0,1.0


# Train Test Split

In [50]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['review/overall','user/gender','review/text','clean_text'])
y = df['review/overall']
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape)

(30000, 510)


# Model Building

In [52]:
from lazypredict.Supervised import LazyRegressor
reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
models,pred = reg.fit(X_train, X_test, y_train, y_test)
models

 24%|███████████████████▌                                                              | 10/42 [03:56<20:47, 38.98s/it]

GammaRegressor model failed to execute
Some value(s) of y are out of the valid range of the loss 'HalfGammaLoss'.


 76%|██████████████████████████████████████████████████████████████▍                   | 32/42 [24:05<06:22, 38.25s/it]

QuantileRegressor model failed to execute
Solver interior-point is not anymore available in SciPy >= 1.11.0.


 98%|████████████████████████████████████████████████████████████████████████████████  | 41/42 [35:44<00:49, 49.55s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016946 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 122543
[LightGBM] [Info] Number of data points in the train set: 30000, number of used features: 510
[LightGBM] [Info] Start training from score 3.888667


100%|██████████████████████████████████████████████████████████████████████████████████| 42/42 [35:45<00:00, 51.09s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
LGBMRegressor,0.68,0.70,0.39,1.63
HistGradientBoostingRegressor,0.68,0.70,0.39,10.56
ElasticNetCV,0.67,0.69,0.39,2.55
LassoCV,0.67,0.69,0.39,2.59
LarsCV,0.67,0.69,0.39,2.53
LassoLarsCV,0.67,0.69,0.39,2.53
LassoLarsIC,0.67,0.69,0.39,1.04
GradientBoostingRegressor,0.67,0.69,0.39,66.84
OrthogonalMatchingPursuit,0.67,0.69,0.39,0.48


In [ ]:
best_model = RandomForestRegressor(random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)
print(f"RMSE: {rmse}")
print(f"R²: {r2}")
new_data = X_test[:5]
predictions = best_model.predict(new_data)
print("Predictions:", predictions)